<a href="https://colab.research.google.com/github/KacperGrodecki/image_proccessing/blob/main/kuba_img_text_reading_found_words_analyzing_enchant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pyenchant

In [3]:
!apt install -qq enchant

enchant is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [4]:
!apt-get install myspell-pl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
myspell-pl is already the newest version (20170707-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [5]:
import pandas as pd
import cv2
import numpy as np
import re
import matplotlib.pyplot as plt      
import pickle
from os import listdir
from os.path import isfile, join
import gc
import sys
import pickle
import string
import enchant

In [6]:
d = enchant.Dict('PL')
d.check("guzik") # False



True

In [7]:
text=pd.read_csv('drive/MyDrive/kuba/odczyt_0_85.csv',index_col=0)

In [8]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [25]:
def word_correction_enchant(a):
    a=str(a)
    correct=''
    for word in a.split():
        if len(word)>1:
            if word.isnumeric():
              correct+=word+' '
              continue
            try:
              if int(d.check(word))==1:
                correct+=word+' '
            except:
              pass     
    return correct

In [26]:
correct=text.applymap(lambda x: word_correction_enchant(x))

In [40]:
correct.head()

,0
0,
1,Własność archiwum Tytuł teczki Daty skrajne je...
2,IPN BU
3,KARTA INFORMACYJNA
4,Instytut Pamięci Narodowej Warszawie


In [42]:
correct = correct[correct['0']!= '']

In [46]:
correct

,0
1,Własność archiwum Tytuł teczki Daty skrajne je...
2,IPN BU
3,KARTA INFORMACYJNA
4,Instytut Pamięci Narodowej Warszawie
6,MSW
...,...
13682,środo katolików świeckich
13683,rozkład spowodowany brakiem pro-
13684,działania. Świadczy to bezsensowności istnienia
13685,tzw. trójpolówki katolickiej
